In [3]:
#Расчет мощности теста

import numpy as np
from statsmodels.stats.power import TTestIndPower

# Параметры для расчета
effect_size = 0.2  # Размер эффекта (разница между группами, деленная на стандартное отклонение)
alpha = 0.05  # Уровень значимости (вероятность ошибки первого рода)
power = 0.8  # Желаемая мощность теста
ratio = 1  # Соотношение размеров групп, 1 если равны

# Создаем объект для расчета мощности
analysis = TTestIndPower()

# Расчет размера выборки
sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=ratio)
print(f'Необходимый размер выборки для каждой группы: {np.ceil(sample_size)}')

# Расчет мощности теста при заданном размере выборки
# Предположим, у нас есть выборка размером 500 в каждой группе
sample_size_per_group = 15
calculated_power = analysis.solve_power(effect_size=effect_size, nobs1=sample_size_per_group, alpha=alpha, ratio=ratio)
print(f'Мощность теста при размере выборки 500 в каждой группе: {calculated_power}')

Необходимый размер выборки для каждой группы: 394.0
Мощность теста при размере выборки 500 в каждой группе: 0.08264675719194485


In [2]:
#Расчет MDE для АБ теста

import numpy as np
from statsmodels.stats.power import TTestIndPower

# Параметры для расчета
alpha = 0.05  # Уровень значимости
power = 0.8  # Желаемая мощность теста
sample_size_per_group = 500  # Размер выборки в каждой группе
ratio = 1  # Соотношение размеров групп, 1 если равны

# Создаем объект для расчета мощности
analysis = TTestIndPower()

# Расчет MDE (Minimum Detectable Effect)
mde = analysis.solve_power(effect_size=None, nobs1=sample_size_per_group, alpha=alpha, power=power, ratio=ratio)
print(f'Минимально обнаруживаемый эффект (MDE): {mde}')

Минимально обнаруживаемый эффект (MDE): 0.17735842307242328


In [ ]:
#P-value показывает вероятность получить наблюдаемые (или более экстремальные) результаты при условии, что нулевая гипотеза верна.

In [ ]:
#Разделения на группы для АБ теста

### 1. Рандомизация

Это самый базовый и часто используемый метод разбиения, при котором пользователи случайным образом назначаются к контрольной 
или тестовой группе. 
Этот метод помогает убедиться, что между группами нет систематических различий, кроме варьируемого фактора.

import numpy as np

# Количество пользователей
num_users = 100

# Генерация случайных групп
groups = np.random.choice(['A', 'B'], size=num_users)

print(groups)
    
### 2. Стратификация

При стратификации пользователи сначала делятся на подгруппы (страты) 
на основе определенных характеристик (например, возраста, пола, географического расположения), 
а затем внутри каждой страты происходит случайное разбиение на тестовую и контрольную группы. 
Этот метод позволяет сделать группы более сопоставимыми по ключевым характеристикам.

import pandas as pd

# Создание датафрейма с информацией о пользователях
data = pd.DataFrame({
    'user_id': range(1, 101),
    'age': np.random.randint(18, 55, size=100),
})

# Стратификация по возрасту
data['age_group'] = pd.cut(data['age'], bins=[17, 24, 34, 44, 54], labels=['18-24', '25-34', '35-44', '45-54'])
data['group'] = data.groupby('age_group').transform(lambda x: np.random.choice(['A', 'B'], size=len(x)))

print(data)


### 3. Блочное разбиение

При блочном разбиении все пользователи разделяются на блоки фиксированного размера, 
внутри которых происходит равномерное распределение по группам. 
Этот метод обеспечивает равное количество участников в каждой группе и часто используется, когда важно контролировать размеры групп.

# Предполагаем, что `data` уже создан и содержит 100 пользователей
block_size = 4

# Генерация блоков
blocks = np.arange(num_users) // block_size
np.random.shuffle(blocks)
data['block'] = blocks

# Разбиение внутри блоков
data['group'] = data.groupby('block').transform(lambda x: np.random.choice(['A', 'B'], size=len(x), replace=True))

print(data)

### 4. Хеширование

Метод хеширования использует алгоритмы хеширования для определения принадлежности пользователя к тестовой или контрольной группе 
на основе уникального идентификатора пользователя (например, email или ID). 
Этот метод обеспечивает повторяемость разбиения и позволяет легко определить группу пользователя без необходимости хранения 
дополнительной информации о разбиении.

import hashlib

def assign_group(user_id, salt='experiment_salt', n_groups=2):
    hash_object = hashlib.sha256((str(user_id) + salt).encode())
    hex_dig = hash_object.hexdigest()
    return int(hex_dig, 16) % n_groups

# Пример использования
user_ids = range(1, 101)
groups = ['A' if assign_group(user_id) == 0 else 'B' for user_id in user_ids]

print(groups)

    
### 5. Платформенно-зависимое разбиение

В некоторых случаях разбиение может зависеть от платформы, на которой проводится тест 
(например, разные версии мобильного приложения для iOS и Android). 
Это позволяет учитывать специфику платформы при анализе результатов.

# Предполагаем, что у нас есть датафрейм с информацией о платформе пользователя
data = pd.DataFrame({
    'user_id': range(1, 101),
    'platform': np.random.choice(['iOS', 'Android'], size=100),
})

# Разбиение по платформе
data['group'] = data['platform'].map(lambda x: 'A' if x == 'iOS' else 'B')

print(data)

### 6. Предварительное разбиение

При предварительном разбиении пользователи делятся на группы до начала теста на основе определенных критериев (например, истории покупок).
Этот метод может быть полезен, когда необходимо тестировать воздействие на определенную целевую аудиторию.

### Важные аспекты при разбиении:

Балансировка групп: Важно стремиться к тому, чтобы группы были максимально сбалансированы по ключевым характеристикам, 
чтобы минимизировать влияние внешних факторов.
Размер групп: Размеры групп должны быть достаточными для обеспечения статистической мощности теста.
Повторяемость: Метод разбиения должен обеспечивать возможность повторения теста с теми же участниками в каждой группе, если это необходимо.

Выбор метода разбиения зависит от конкретных целей и условий проведения A/B теста, а также от доступных данных о пользователях.

In [ ]:
#Методы оценки аб тестов

### 1. **T-тест (t-test)**

T-тест является одним из наиболее часто используемых методов для сравнения средних значений двух групп. 
Он может быть применен, когда данные распределены нормально. 
Существуют различные варианты t-теста, включая одновыборочный, независимый двухвыборочный и связанный двухвыборочный t-тесты.

### 2. **Z-тест (z-test)**

Z-тест используется для сравнения средних двух групп, когда известны стандартные отклонения генеральных совокупностей 
и размер выборки достаточно большой. Этот тест часто применяется для сравнения пропорций.

### 3. **Анализ дисперсии (ANOVA)**

ANOVA позволяет сравнивать средние значения более чем двух групп. 
Однофакторный ANOVA используется, когда необходимо оценить влияние одного фактора, в то время как 
многофакторный ANOVA применяется для анализа влияния двух или более факторов.

### 4. **Манна-Уитни (Mann-Whitney U test)**

Этот непараметрический тест используется для сравнения двух независимых выборок. 
Он является альтернативой t-тесту для случаев, когда данные не распределены нормально.

### 5. **Хи-квадрат (Chi-squared test)**

Тест хи-квадрат применяется для сравнения категориальных переменных. 
Он может использоваться для проверки гипотезы о наличии или отсутствии статистически значимых различий в распределении категориальных переменных между группами.

### 6. **Логистическая регрессия**

Логистическая регрессия часто используется для анализа данных A/B тестов, 
когда зависимая переменная является бинарной (например, конверсия: да/нет). Этот метод позволяет оценить влияние одной или нескольких независимых переменных на вероятность наступления события.

### Примеры на Python

#### T-тест с использованием SciPy:

python
from scipy import stats

# Допустим, у нас есть результаты двух групп
group_a = [10, 12, 9, 11, 12, 9]
group_b = [11, 12, 13, 14, 15, 13]

# Выполнение t-теста
t_stat, p_value = stats.ttest_ind(group_a, group_b)
print(f"T-statistic: {t_stat}, P-value: {p_value}")


#### Манна-Уитни с использованием SciPy:

python
from scipy import stats

# Допустим, у нас есть результаты двух групп
group_a = [10, 12, 9, 11, 12, 9]
group_b = [11, 12, 13, 14, 15, 13]

# Выполнение теста Манна-Уитни
u_stat, p_value = stats.mannwhitneyu(group_a, group_b)
print(f"U-statistic: {u_stat}, P-value: {p_value}")


Выбор метода оценки зависит от типа данных, распределения и целей исследования. 
Важно также учитывать размер выборки и предположения, лежащие в основе каждого статистического теста.

In [ ]:
#Виды метрик в АБ тесте

